In [ ]:
import oracledb
from google.cloud.bigquery import Client
import os, json
from google.cloud import secretmanager
import pandas as pd

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user= os.getenv('DB_USER'),
    password= os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    project_id = os.getenv('BS_PROJECT_ID'),
    table_uri = os.getenv('BS_TABLE_URI'),  
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data_from_table():
    PROJECT_ID = oracle_secrets['project_id'] 
    TABLE_URI = oracle_secrets['table_uri'] 

    client = Client(project=PROJECT_ID)
    job = client.query(f"SELECT * FROM `{TABLE_URI}`")
       
    df = job.to_dataframe().drop_duplicates()
    
    # some data cleaning
    
    char_to_replace = {'≥' : '>=', '≤' : '<='}
    for old, new in char_to_replace.items():
        df['sats_beskrivelse'] = df['sats_beskrivelse'].str.replace(old, new)
    
    return df

In [ ]:
def send_data():

    # Henter data fra BQ tabellen
    df = get_data_from_table()
    df['opprettet'] = pd.to_datetime(df.opprettet).dt.tz_localize(None).astype(str)
    df['periode'] = ( pd.to_datetime(df['bestillingsdato'], errors='coerce').dt.strftime('%Y%m').where(lambda x: x.notna(), None) )
    #pd.to_datetime(df['bestillingsdato']).dt.strftime('%Y%m').astype(str)
    
    user = oracle_secrets['user'] + '[DVH_FAM_HM]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            
            delete_sql = """DELETE FROM brillestonad"""
            cursor.execute(delete_sql)
            conn.commit()
            
            
            if len(df) > 0:
                rows = [tuple(x) for x in df.values]
                cursor.executemany('''INSERT INTO brillestonad (ID,FNR_BARN,FNR_INNSENDER,ORGNR,BESTILLINGSDATO,BRILLEPRIS,BESTILLINGSREFERANSE,
                                        BEHANDLINGSRESULTAT,SATS,SATS_BELOP,SATS_BESKRIVELSE,BELOP,OPPRETTET,KILDE,PERIODE) 
                                         VALUES (:1,:2,:3,:4,:5,nvl(round(to_number(:6),2),0),:7,:8,:9,nvl(round(to_number(:10),2),0),:11,nvl(round(to_number(:12),2),0),TO_TIMESTAMP(:13, 'yyyy-mm-dd HH24:MI:SS.FF6'),:14,:15)''',rows)
                conn.commit()

            sql_fk_person1 = """
                MERGE INTO DVH_FAM_HM.brillestonad T
                USING (
                WITH brille as (
                SELECT 
                A.FNR_BARN,
                A.FNR_INNSENDER,
                A.pk_fam_brillestonad,
                A.opprettet,
                to_char(a.BESTILLINGSDATO,'YYYYMM') periode_NY,
                case when barn.skjermet_kode = 0 then BARN.FK_PERSON1
                    when barn.skjermet_kode != 0 then -1
                    else null
                    end FK_PERSON1_BARN_NY,
                case when INNSENDER.skjermet_kode = 0 then INNSENDER.FK_PERSON1
                    when INNSENDER.skjermet_kode != 0 then -1
                    else null
                    end FK_PERSON1_INNSENDER_NY,
                case when dim_person_innsender.skjermet_kode = 0 then dim_person_innsender.pk_dim_person
                    when dim_person_innsender.skjermet_kode != 0 then -1
                    else null 
                    end FK_DIM_PERSON_INNSENDER_NY,
                case when dim_person_barn.skjermet_kode = 0 then dim_person_barn.pk_dim_person
                    when dim_person_barn.skjermet_kode != 0 then -1
                    else null 
                    end FK_DIM_PERSON_barn_NY

                FROM brillestonad A

                left outer join dt_person.ident_off_id_til_fk_person1 BARN ON
                BARN.OFF_ID=FNR_BARN AND
                BARN.GYLDIG_FRA_DATO<OPPRETTET AND
                BARN.GYLDIG_TIL_DATO>=OPPRETTET
                left outer join dt_person.ident_off_id_til_fk_person1 INNSENDER ON
                INNSENDER.OFF_ID=FNR_INNSENDER AND
                INNSENDER.GYLDIG_FRA_DATO<OPPRETTET AND
                INNSENDER.GYLDIG_TIL_DATO>=OPPRETTET
                left outer join dt_person.dim_person dim_person_innsender ON
                dim_person_innsender.fk_person1 = innsender.fk_person1 AND
                dim_person_innsender.gyldig_fra_dato < OPPRETTET AND
                dim_person_innsender.gyldig_til_dato >= OPPRETTET
												  
                left outer join dt_person.dim_person dim_person_barn ON
                dim_person_barn.fk_person1 = barn.fk_person1 AND
                dim_person_barn.gyldig_fra_dato < OPPRETTET AND
                dim_person_barn.gyldig_til_dato >= OPPRETTET
                
                ) 
                select * from brille)  S
                 
                ON ( S.PK_FAM_BRILLESTONAD=T.PK_FAM_BRILLESTONAD)    
                WHEN MATCHED
                THEN UPDATE
                     SET 
                        T.FK_PERSON1_BARN=S.FK_PERSON1_BARN_NY,
                        T.FK_PERSON1_INNSENDER=S.FK_PERSON1_INNSENDER_NY,
                        T.FK_DIM_PERSON_INNSENDER=s.FK_DIM_PERSON_INNSENDER_NY,
                        T.FK_DIM_PERSON_BARN=s.FK_DIM_PERSON_BARN_NY,
                        T.FNR_BARN=CASE WHEN S.FK_PERSON1_BARN_NY IS NOT NULL OR S.FK_PERSON1_BARN_NY != -1 THEN NULL ELSE T.FNR_BARN END,
                        T.FNR_innsender=CASE WHEN S.FK_PERSON1_innsender_NY IS NOT NULL OR S.FK_PERSON1_innsender_NY != -1 THEN NULL ELSE T.FNR_innsender END,
                        T.oppdatert_dato=localtimestamp
            """
            
            cursor.execute(sql_fk_person1)
            conn.commit()

            sql_delete_k76 = """delete from DVH_FAM_HM.brillestonad where FK_PERSON1_BARN = -1"""
            cursor.execute(sql_delete_k76)
            conn.commit()

            sql_barn_info = """
                merge into brillestonad b
                    using
                    (
                        select
                            b2.pk_fam_brillestonad,
                            case
                                when dim_barn.fodt_dato is not null
                                then floor(months_between(b2.bestillingsdato, dim_barn.fodt_dato) / 12)
                            end alder_barn,
                            case
                                when dim_barn.kjonn_nr = 1 then 'M'
                                when dim_barn.kjonn_nr = 0 then 'K'
                                else 'U'
                            end as kjonn_barn,
                            dim_barn.fk_dim_geografi_bosted as fk_dim_geografi
                        from brillestonad b2
                        left join dt_person.dim_person dim_barn
                        on dim_barn.fk_person1 = b2.fk_person1_barn
                        --and dim_barn.gyldig_fra_dato is not null
                        --and dim_barn.gyldig_til_dato is not null
                        and b2.bestillingsdato between dim_barn.gyldig_fra_dato and dim_barn.gyldig_til_dato
                        where b2.bestillingsdato > DATE '0001-01-01'

                    ) src
                    on
                    (
                        b.pk_fam_brillestonad = src.pk_fam_brillestonad
                    )
                    when matched then
                    update set
                        b.alder_barn = src.alder_barn,
                        b.kjonn_barn = src.kjonn_barn,
                        b.fk_dim_geografi_barn = src.fk_dim_geografi
            """
            cursor.execute(sql_barn_info)
            conn.commit()
            
            sql_update_k76 = """
                            UPDATE DVH_FAM_HM.brillestonad
                            SET ORGNR = null
                            WHERE FK_PERSON1_INNSENDER = -1
                        """
            cursor.execute(sql_update_k76)
            conn.commit()

In [ ]:
send_data()